In [33]:
# mostly torch imports and plot imports
import os
import sys
import cv2
# mostly torch imports and plot imports
import torch
import shutil
import glob
import pickle
import random
random.seed(42)
# import colorama
# from colorama import Fore, Style
import numpy as np
np.random.seed(42)
import torch.utils
import torchvision
from torch import optim
import torch.distributions
torch.manual_seed(42)
from tqdm import tqdm
import torch.nn as nn
from torchsummary import summary
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
from matplotlib import rc, rcParams
from numpy import sin

use_cuda = torch.cuda.is_available()
print('use_cuda: {}'.format(use_cuda))
device = torch.device("cuda" if use_cuda else "cpu")
print("Device to be used : ",device)

use_cuda: True
Device to be used :  cuda


In [47]:
# GPU operations have a separate seed we also want to set
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

# Additionally, some operations on a GPU are implemented stochastic for efficiency
# We want to ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [48]:

class ClassificationNetwork(nn.Module):
    def __init__(self):
        super(ClassificationNetwork, self).__init__()
        self.fc1 = nn.Linear(10, 2)
        self.fc2 = nn.Linear(2,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Create an instance of the network
model = ClassificationNetwork()

In [49]:
class DataGenerator(Dataset):
    def __init__(self, X_VALUES, Y_VALUES, choice):
        self.X = X_VALUES
        self.Y = Y_VALUES
        self.choice = choice

    def __get__number__(self, str_num):
        return np.fromiter((int(bit) for bit in str_num), dtype=np.int8)

    # NUMBER OF FILES IN THE DATASET
    def __len__(self):
        return len(self.X)

    # GETTING SINGLE PAIR OF DATA
    def __getitem__(self, idx):
        # print("X = ",self.X)
        # print("idx = ",idx)
        X_numpy = self.__get__number__(self.X[idx])
        # X_numpy_reshaped = X_numpy.reshape((1, 10))
        # print("X_numpy_reshaped = ", torch.FloatTensor(X_numpy_reshaped))
        # print("Y = ", torch.FloatTensor(self.Y[idx]))
        # return torch.FloatTensor(X_numpy_reshaped), torch.FloatTensor(self.Y[idx])
        return torch.FloatTensor(X_numpy), self.Y[idx] #torch.FloatTensor(self.Y[idx])



In [50]:
def load_data(X_VALUES, Y_VALUES, choice="train", batch_size=2, num_workers=10, shuffle=True):
    dataset = DataGenerator(X_VALUES, Y_VALUES, choice=choice)
    data_loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)
    return data_loader


# save checkpoint in pytorch
def save_ckp(checkpoint, checkpoint_path):
    torch.save(checkpoint, checkpoint_path)


# load checkpoint in pytorch
def load_ckp(checkpoint_path, model, model_opt):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    model_opt.load_state_dict(checkpoint['optimizer'])
    return model, model_opt, checkpoint['epoch']

In [51]:
def train_epoch(train_loader, model, optimizer, epoch):
    print("\n\n---------------------------------------------------------------------------------------------------------------\n")

    progress_bar = tqdm(enumerate(train_loader))
    total_loss = 0.0

    N = 0

    for step, (x, y) in progress_bar:
        # if mask_1 is None and low_res_img is None:
        #     continue
        model.train()

        #TRANSFERRING DATA TO DEVICE
        x = x.to(device)
        y = y.to(device)

        # clear the gradient
        optimizer.zero_grad()

        #GETTING THE PREDICTED IMAGE
        # print("x shape == ",x.shape)
        pred_y = model.forward(x)

        # print ("**************")
        # print (pred_y.shape)
        y = y.unsqueeze(-1)
        # print (y.shape)
        # print ("$$$$$$$$$$$$$$$")
        y = y.float()

        #LOSS FUNCTIONS
        BCELOSS = nn.BCELoss()

        #CALCULATING LOSSES
        BCE_loss = BCELOSS(pred_y, y)

        # print (f'Prediction: {pred_y} | {y}')

        #LOSS TAKEN INTO CONSIDERATION
        loss = BCE_loss


        # CALCULATING METRICS
        total_loss += loss

        # print(loss)

        #BACKPROPAGATING THE LOSS
        loss.backward()
        optimizer.step()

        #DISPLAYING THE LOSS
        # progress_bar.set_description("Epoch: {} -  Loss: {} ".format(epoch, loss))

        N += x.shape[0]


    # with open("history/train_logs.txt", "a") as text_file:
    #     text_file.write("{} {}\n".format(epoch, total_loss))

    # print("Training Epoch: {} |  Total Loss: {} | Total Dice: {} | Total Jaccard: {} | N: {}".format(epoch,total_loss, total_dice, total_jacard,N))
    print("Training Epoch: {} |  Loss: {}".format(epoch, total_loss/N))

    return model, optimizer



def test_epoch(test_loader, model, optimizer, epoch):

    progress_bar = tqdm(enumerate(test_loader))
    total_loss = 0.0

    #SETTING THE NUMBER OF IMAGES TO CHECK AFTER EACH ITERATION
    no_img_to_write = 20

    total_loss = 0.0

    N = 0
    for step, (x, y) in progress_bar:

        x = x.to(device)
        y = y.to(device)


        #PREDICTED IMAGE
        pred_y = model.forward(x)

        #LOSS FUNCTIONS
        BCELOSS = nn.BCELoss()

        model.eval()

        y = y.unsqueeze(-1)
        y = y.float()

        #CALCULATING LOSSES
        BCE_loss = BCELOSS(pred_y, y)

        #LOSS TAKEN INTO CONSIDERATION
        loss = BCE_loss

        # CALCULATING METRICS
        total_loss += loss

        N += x.shape[0]


        # progress_bar.set_description("Epoch: {} -  Loss: {} ".format(epoch, total_loss))


    # with open("history/test_logs.txt", "a") as text_file:
    #     text_file.write("{} {}\n".format(epoch, total_loss))

    print("Test Epoch: {} |  Loss: {}".format(epoch, total_loss/N))
    print("---------------------------------------------------------------------------------------------------------------")



In [52]:
def train_val_test(train_loader, test_loader, model, optimizer, n_epoch, resume):

    #PATH TO SAVE THE CHECKPOINT
    checkpoint_path = "checkpoint/classification_net.pt"

    epoch = 0
    #IF TRAINING IS TO RESUMED FROM A CERTAIN CHECKPOINT
    if resume:
        model, optimizer, epoch = load_ckp(
            checkpoint_path, model, optimizer)

    while epoch <= n_epoch:
        epoch += 1
        model, optimizer = train_epoch(train_loader, model, optimizer, epoch)

        #CHECKPOINT CREATION
        checkpoint = {'epoch': epoch+1, 'state_dict': model.state_dict(),
                      'optimizer': optimizer.state_dict()}

        #CHECKPOINT SAVING
        save_ckp(checkpoint, checkpoint_path)
        print("Checkpoint Saved")

    print("************************ Final Test Epoch *****************************")

    with torch.no_grad():
        test_epoch(test_loader, model, optimizer, epoch)

In [53]:
def is_pal(X):
    return X == X[::-1]

def generate_10_bit_binary_numbers():
    binary_numbers = []

    for i in range(2**10):
        binary_str = format(i, '010b')
        binary_numbers.append(binary_str)

    return binary_numbers


# Generate and store all 10-bit binary numbers
binary_numbers_list = generate_10_bit_binary_numbers()


Y_VALUES = []
X_VALUES = []

# Print and/or use the generated binary numbers as needed
for binary_number in binary_numbers_list:
    #print(binary_number)
    if is_pal(binary_number) == True:
        X_VALUES.append(binary_number)
        Y_VALUES.append(1)
    else:
        X_VALUES.append(binary_number)
        Y_VALUES.append(0)

In [54]:
! mkdir checkpoint

mkdir: cannot create directory ‘checkpoint’: File exists


In [55]:
! mkdir history

mkdir: cannot create directory ‘history’: File exists


In [74]:

print ('modified!')

train_till = int(0.8*len(X_VALUES))

X_VALUES_TRAIN = X_VALUES[:train_till]
Y_VALUES_TRAIN = Y_VALUES[:train_till]

X_VALUES_TEST = X_VALUES[train_till:]
Y_VALUES_TEST = Y_VALUES[train_till:]

print("Total Number of Training data : ", len(X_VALUES_TRAIN))
print("Total Number of Testing data : ", len(X_VALUES_TEST))


# CREATING THE TRAIN LOADER
train_loader = load_data(
    X_VALUES_TRAIN, Y_VALUES_TRAIN, choice="train", batch_size=16, num_workers=1, shuffle=True)

# #CREATING THE TEST LOADER
test_loader = load_data(
    X_VALUES_TEST, Y_VALUES_TEST, choice="test", batch_size=1, num_workers=1, shuffle=False)

#CALLING THE MODEL
model = ClassificationNetwork()
model = model.to(device)

summary(model, input_size=(1, 10))

#DEFINING THE OPTIMIZER
optimizer = optim.Adam(
    [p for p in model.parameters() if p.requires_grad], lr=5e-03, weight_decay=5e-4)

n_epoch = 200

#INDICATOR VARIABLE TO RESUME TRAINING OR START AFRESH
resume = False
train_val_test(train_loader, test_loader, model, optimizer, n_epoch, resume)





modified!
Total Number of Training data :  819
Total Number of Testing data :  205
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 2]              22
            Linear-2                 [-1, 1, 1]               3
           Sigmoid-3                 [-1, 1, 1]               0
Total params: 25
Trainable params: 25
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


---------------------------------------------------------------------------------------------------------------



52it [00:00, 268.88it/s]


Training Epoch: 1 |  Loss: 0.02894730493426323
Checkpoint Saved


---------------------------------------------------------------------------------------------------------------



52it [00:00, 257.21it/s]

Training Epoch: 2 |  Loss: 0.012304768897593021
Checkpoint Saved


---------------------------------------------------------------------------------------------------------------



...

Training Epoch: 200 |  Loss: 0.007965056225657463
Checkpoint Saved


---------------------------------------------------------------------------------------------------------------




52it [00:00, 111.74it/s]

Training Epoch: 201 |  Loss: 0.007995430380105972
Checkpoint Saved
************************ Final Test Epoch *****************************



205it [00:00, 243.46it/s]

Test Epoch: 201 |  Loss: 0.17940735816955566
---------------------------------------------------------------------------------------------------------------


### Calculating Precision

In [76]:
def precision(result, reference):
    result = np.atleast_1d(result.astype(np.bool_))
    reference = np.atleast_1d(reference.astype(np.bool_))

    tp = np.count_nonzero(result & reference)
    fp = np.count_nonzero(result & ~reference)

    try:
        precision = tp / float(tp + fp)
    except ZeroDivisionError:
        precision = 0.0

    return precision

In [90]:
# Calculating Precision
ALL_PREDS_VALUES = []
ALL_Y_VALUES = []

progress_bar = tqdm(enumerate(test_loader))
total_loss = 0.0

N = 0
for step, (x, y) in progress_bar:
    x = x.to(device)
    y = y.to(device)


    #PREDICTED IMAGE
    pred_y = model.forward(x)

    #LOSS FUNCTIONS
    BCELOSS = nn.BCELoss()

    model.eval()

    y = y.unsqueeze(-1)
    y = y.float()
    threshold = 0.5
    if pred_y.item() > threshold:
        ALL_PREDS_VALUES.append(1)
    else:
        ALL_PREDS_VALUES.append(0)
    ALL_Y_VALUES.append(int(y.item()))

ALL_Y_VALUES = np.array(ALL_Y_VALUES)
ALL_PREDS_VALUES = np.array(ALL_PREDS_VALUES)

205it [00:00, 439.76it/s]


In [91]:
ALL_PREDS_VALUES

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

In [93]:
precision_val = precision(ALL_PREDS_VALUES, ALL_Y_VALUES)
print("precision ==> ",precision_val)

precision ==>  0.0


### Checking hidden layer weights

In [65]:
model

ClassificationNetwork(
  (fc1): Linear(in_features=10, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [68]:
model.fc1.weight

Parameter containing:
tensor([[-0.2160,  0.0603, -0.1676, -0.1468,  0.2727,  0.1527, -0.0865,  0.0465,
          0.0613,  0.0741],
        [-0.1082, -0.1862,  0.2094,  0.0425,  0.1324, -0.1198,  0.1532,  0.2301,
          0.1923,  0.0860]], requires_grad=True)

In [69]:
model.fc2.weight

Parameter containing:
tensor([[0.1505, 0.6718]], requires_grad=True)